# ATEK CoLab Notebook

Welcome to ATEK CoLab Notebook. This notebook
 will walk through the steps of preparing an Aria data sequence with annotations ([AriaDigitalTwin (ADT)](https://www.projectaria.com/datasets/adt/)), for use in a 3D object detection ML task.
We will go through the following steps:
1. downloading ADT sample data
2. preprocess ADT sample data
3. visualize the preprocessed data
4. run model inference with ATEK preprocessed data
5. evaluate model performance.

## Environment set up
Click restart session when being asked. Then run the following code block again.

In [ ]:
!pip install 'git+https://github.com/YLouWashU/omni3d.git'
!pip install projectaria-atek==1.0.0
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install iopath
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
!pip install rerun-sdk[notebook]

  Cloning https://github.com/YLouWashU/omni3d.git to /tmp/pip-req-build-lyp4imqh
  Running command git clone --filter=blob:none --quiet https://github.com/YLouWashU/omni3d.git /tmp/pip-req-build-lyp4imqh
  Resolved https://github.com/YLouWashU/omni3d.git to commit a5bcbf8fd45e9f69e7ec9de1cebea9e82226d6e2
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-mwvb7a7a
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-mwvb7a7a
  Resolved https://github.com/facebookresearch/detectron2.git to commit 8d85329aed8506ea3672e3e208971345973ea761
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.2 MB/s eta 0:00:00


In [1]:
import requests

url = "https://www.projectaria.com/async/sample/download/?bucket=atek&filename=ATEK_example_model_weights.tar"
response = requests.get(url)

# Specify the path where you want to save the file
file_path = "/content/ATEK_example_model_weights.tar"

# Open the file in binary write mode and write the contents of the response
with open(file_path, "wb") as file:
    file.write(response.content)

In [2]:
!mkdir -p /content/data
!tar -xvf /content/ATEK_example_model_weights.tar -C /content/data

./model_weights/
./model_weights/LICENSE
./model_weights/ATEK_example_cubercnn_weights_trained_on_ADT/
./model_weights/ATEK_example_cubercnn_weights_trained_on_ADT/config.yaml
./model_weights/ATEK_example_cubercnn_weights_trained_on_ADT/metrics.json
./model_weights/ATEK_example_cubercnn_weights_trained_on_ADT/model_final.pth
./model_weights/ATEK_example_cubercnn_weights_trained_on_ADT/last_checkpoint
./model_weights/ATEK_example_cubercnn_weights_trained_on_ASE/
./model_weights/ATEK_example_cubercnn_weights_trained_on_ASE/config.yaml
./model_weights/ATEK_example_cubercnn_weights_trained_on_ASE/metrics.json
./model_weights/ATEK_example_cubercnn_weights_trained_on_ASE/last_checkpoint
./model_weights/ATEK_example_cubercnn_weights_trained_on_ASE/model_recent.pth


In [3]:
!git clone https://github.com/facebookresearch/ATEK.git

Cloning into 'ATEK'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 184 (delta 42), reused 64 (delta 25), pack-reused 47 (from 1)
Receiving objects: 100% (184/184), 8.53 MiB | 17.71 MiB/s, done.
Resolving deltas: 100% (50/50), done.


## Part1: Data proprocessing

###  Data preprocessing requirements
ADT sequence has:
1. Aria recording (VRS).
2. MPS trajectory file (CSV).
3. Object detection annotation files (3 csv files + 1 json file).

CubeRCNN model needs synchronized data frame containing:
1. Upright RGB camera image.
2. Linear camera calibration matrix.
3. Object bounding box annotations in 2D + 3D.
4. Camera-to-object poses.

**Before ATEK**, users need to implement all the followings to prepare ADT sequence into CubeRCNN model:
1. Parse in ADT sequence data using `projectaria_tools` lib.   
2. Properly synchronize sensor + annotation data into training samples.
3. Perform additional image & data processing:
    1. Undistort image + camera calibration.
    2. Rescale camera resolution.
    3. Rotate image + camera calibration.
    4. Undistort + rescale + rotate object 2D bounding boxes accordingly.

In [ ]:
import faulthandler
import logging
import os
from logging import StreamHandler
import numpy as np
from typing import Dict, List, Optional
import torch
import sys
from itertools import islice
from tqdm import tqdm

from atek.data_preprocess.genera_atek_preprocessor_factory import (
    create_general_atek_preprocessor_from_conf,
)
from atek.viz.atek_visualizer import NativeAtekSampleVisualizer
from atek.data_preprocess.general_atek_preprocessor import GeneralAtekPreprocessor
from atek.data_loaders.atek_wds_dataloader import (
    create_native_atek_dataloader
)

from atek.data_loaders.cubercnn_model_adaptor import (
    cubercnn_collation_fn,
    create_atek_dataloader_as_cubercnn
)
from atek.evaluation.static_object_detection.obb3_csv_io import AtekObb3CsvWriter

from atek.data_preprocess.atek_data_sample import (
    create_atek_data_sample_from_flatten_dict,
)
from cubercnn.config import get_cfg_defaults
from cubercnn.modeling.backbone import build_dla_from_vision_fpn_backbone
from cubercnn.modeling.meta_arch import build_model

from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from omegaconf import OmegaConf
import subprocess

## Download ADT sequences from projectaria.com

In [ ]:
adt_sample_path = "./adt_sample_data"
data_sequence_url = "https://www.projectaria.com/async/sample/download/?bucket=adt&filename=aria_digital_twin_test_data_v2.zip"
command_list = [
    f"mkdir -p {adt_sample_path}",
    # Download sample data
    f'curl -o {adt_sample_path}/adt_sample_data.zip -C - -O -L "{data_sequence_url}"',
    # Unzip the sample data
    f"unzip -o {adt_sample_path}/adt_sample_data.zip -d {adt_sample_path}"
]
sequence_path = f"{adt_sample_path}/Apartment_release_golden_skeleton_seq100_10s_sample_M1292"

# Execute the commands for downloading dataset
for command in command_list:
    subprocess.run(command, shell=True, check=True)

###  Set up data and code paths

In [ ]:
example_adt_data_dir = f"{adt_sample_path}/Apartment_release_golden_skeleton_seq100_10s_sample_M1292"
sequence_name = "Apartment_release_golden_skeleton_seq100_10s_sample_M1292"
atek_src_path = "/content/ATEK"
category_mapping_file = f"{atek_src_path}/data/adt_prototype_to_atek.csv"
atek_preprocess_config_path = f"{atek_src_path}/examples/data/adt_cubercnn_preprocess_config.yaml"
preprocess_conf = OmegaConf.load(atek_preprocess_config_path)
# Take viz conf out of preprocess conf
viz_conf = preprocess_conf.visualizer
del preprocess_conf.visualizer

# Create viz conf for inference viewer
infer_viz_config_path = f"{atek_src_path}/examples/data/infer_viz_conf.yaml"
infer_viz_conf = OmegaConf.load(infer_viz_config_path)

output_wds_path = f"{atek_src_path}/examples/data/wds_output"

### Helper functions

In [ ]:
faulthandler.enable()

# Configure logging to display the log messages in the notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

logger = logging.getLogger()


# -------------------- Helper functions --------------------#
def print_data_sample_dict_content(data_sample, if_pretty: bool = False):
    """
    A helper function to print the content of data sample dict
    """
    logger.info("Printing the content in a ATEK data sample dict: ")
    for key, val in data_sample.items():
        if if_pretty and "#" in key:
            key = key.split("#", 1)[1]

        msg = f"\t {key}: is a {type(val)}, "
        if isinstance(val, torch.Tensor):
            msg += f"\n \t\t\t\t with tensor dtype of {val.dtype}, and shape of : {val.shape}"
        elif isinstance(val, list):
            msg += f"with len of : {len(val)}"
        elif isinstance(val, str):
            msg += f"value is {val}"
        else:
            pass
        logger.info(msg)

def run_command_and_display_output(command):
    # Start the process
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    # Poll process.stdout to show stdout live
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
    rc = process.poll()
    return rc

def create_inference_model(config_file, ckpt_dir, use_cpu_only=False):
    """
    Create the model for inference pipeline, with the model config.
    """
    # Create default model configuration
    model_config = get_cfg()
    model_config.set_new_allowed(True)
    get_cfg_defaults(model_config)

    # add extra configs for data
    model_config.MAX_TRAINING_ATTEMPTS = 3
    model_config.TRAIN_LIST = ""
    model_config.TEST_LIST = ""
    model_config.TRAIN_WDS_DIR = ""
    model_config.TEST_WDS_DIR = ""
    model_config.ID_MAP_JSON = ""
    model_config.OBJ_PROP_JSON = ""
    model_config.CATEGORY_JSON = ""
    model_config.DATASETS.OBJECT_DETECTION_MODE = ""
    model_config.SOLVER.VAL_MAX_ITER = 0
    model_config.SOLVER.MAX_EPOCH = 0

    model_config.merge_from_file(config_file)
    if use_cpu_only:
        model_config.MODEL.DEVICE = "cpu"
    model_config.freeze()

    model = build_model(model_config, priors=None)

    _ = DetectionCheckpointer(model, save_dir=ckpt_dir).resume_or_load(
        model_config.MODEL.WEIGHTS, resume=True
    )
    model.eval()

    return model_config, model

### Set up and run ATEK data preprocessor

In [ ]:
# Create ATEK preprocessor from conf. It will automatically choose which type of sample to build.
atek_preprocessor = create_general_atek_preprocessor_from_conf(
    # [required]
    conf=preprocess_conf,
    raw_data_folder = example_adt_data_dir,
    sequence_name = sequence_name,
    # [optional]
    output_wds_folder=output_wds_path,
    output_viz_file=os.path.join(example_adt_data_dir, "atek_preprocess_viz.rrd"),
    category_mapping_file=category_mapping_file,
)

## Preprocessed ATEK data sample content
* Preprocessing input: VRS + csv + jsons
* Preprocessing output (in memory): ATEK data samples: `Dict[torch.Tensor, str, or Dict]`
* Preprocessing output (on local disk): WebDataset (WDS) tar files.

In [ ]:
atek_data_sample = atek_preprocessor[0]
atek_data_sample_dict = atek_data_sample.to_flatten_dict()
print_data_sample_dict_content(atek_data_sample_dict)

# Loop over all samples, and write valid ones to local tar files.
atek_preprocessor.process_all_samples(write_to_wds_flag=True, viz_flag=False)

## Visualize preprocessed ATEK data sample

If you did not see the visualization, please run the following code block again. Sometimes rerun visualization on colab notebook is not stable.

In [ ]:
tar_file_urls = [os.path.join(output_wds_path, f"shards-000{i}.tar") for i in range(2)]
atek_dataloader = create_native_atek_dataloader(urls = tar_file_urls, batch_size = None, num_workers = 1)
atek_viz = NativeAtekSampleVisualizer(viz_prefix = "notebook atek viz", show_on_notebook = True, conf = viz_conf)
for atek_sample in tqdm(islice(atek_dataloader, 10)):
    atek_viz.plot_atek_sample_as_dict(atek_sample)

# Part 2: Run Object detection inference using pre-trained CubeRCNN model
In this example, we demonstrate how to run model inference with preprocessed ATEK data streamed from Data Store.

### Create PyTorch DataLoader, converted to CubeRCNN format
User can add a data transform function from ATEK format -> CubeRCNN format:
1. Dict key remapping.
2. Tensor reshaping & reordering.
3. Other data transformations.

Example data transform function for CubeRCNN model: [src code](https://www.internalfb.com/code/fbsource/[a5c3831c045bc718862d1c512e84d4ed6f79d722]/fbcode/surreal/data_services/atek/atek/data_loaders/cubercnn_model_adaptor.py?lines=44-74)

In [ ]:
import logging
logger = logging.getLogger()
logger.info(
    "-------------------- ATEK WDS data can loaded into Model-specific format --------------- "
)
# The CubeRCNN ModelAdaptor class is wrapped in this function
cubercnn_dataloader = create_atek_dataloader_as_cubercnn(urls = tar_file_urls, batch_size = 1, num_workers = 1)
first_cubercnn_sample = next(iter(cubercnn_dataloader))
logger.info(f"Loading WDS into CubeRCNN format, each sample contains the following keys: {first_cubercnn_sample[0].keys()}")

### Run model inference

In [ ]:
from tqdm import tqdm
model_ckpt_path = "/content/data/model_weights/ATEK_example_cubercnn_weights_trained_on_ADT"
# load pre-trained CubeRCNN model
model_config_file = os.path.join(model_ckpt_path, "config.yaml")
conf = OmegaConf.load(model_config_file)

# setup config and model
model_config, model = create_inference_model(
    model_config_file, model_ckpt_path, use_cpu_only = True
)

# Cache inference results for visualization
input_output_data_pairs = []

# Loop over created Pytorch Dataloader, only 5 batches for demonstration
with torch.no_grad():
    for cubercnn_input_data in tqdm(
       islice(cubercnn_dataloader, 5),
        desc="Inference progress: ",
    ):
        cubercnn_model_output = model(cubercnn_input_data)

        # cache inference results for visualization
        input_output_data_pairs.append((cubercnn_input_data, cubercnn_model_output))

logger.info("Inference completed.")

## Visualize inference result

If you did not see the visualization, please run the following code block again. Sometimes rerun visualization on colab notebook is not stable.

In [ ]:
from atek.viz.cubercnn_visualizer import CubercnnVisualizer

# Visualize cached inference results
logger.info("Visualizing inference results.")
cubercnn_visualizer = CubercnnVisualizer(viz_prefix = "inference_visualizer", show_on_notebook = True, conf = viz_conf)
for input_data_as_list, output_data_as_list in input_output_data_pairs:
    for single_cubercnn_input, single_cubercnn_output in zip(input_data_as_list, output_data_as_list):
        timestamp_ns = single_cubercnn_input["timestamp_ns"]
        # Plot RGB image
        cubercnn_visualizer.plot_cubercnn_img(single_cubercnn_input["image"], timestamp_ns = timestamp_ns)

        # Plot GT and prediction in different colors
        single_cubercnn_output["T_world_camera"] = single_cubercnn_input["T_world_camera"] # This patch is needed for visualization
        cubercnn_visualizer.plot_cubercnn_dict(cubercnn_dict = single_cubercnn_input, timestamp_ns = timestamp_ns, plot_color = cubercnn_visualizer.COLOR_GREEN, suffix = "_model_input")
        cubercnn_visualizer.plot_cubercnn_dict(cubercnn_dict = single_cubercnn_output, timestamp_ns = timestamp_ns, plot_color = cubercnn_visualizer.COLOR_RED, suffix = "_model_output")

## Part 3: Evaluate model performance
ATEK provides **per-task**:
1. Standardized prediction file formats.
2. Lib for common eval metrics.
3. Benchmarking scripts.

Example prediction file format for 3D object detection:

| time_ns       | tx_world_object | ty_world_object | tz_world_object | qw_world_object | qx_world_object | qy_world_object | qz_world_object | scale_x | scale_y | scale_z | name    | instance | sem_id | prob    |
|---------------|-----------------|-----------------|-----------------|-----------------|-----------------|-----------------|-----------------|---------|---------|---------|---------|----------|--------|--------|
| 14588033546600| -4.119894       | 0.986124        | 2.796770        | 0.008052        | -0.022706       | -0.010150       | 0.999658        | 0.190   | 2.146   | 0.900   | door    | -1       | 32     | 0.994462|
| 14588033546600| -3.875954       | 0.837941        | 4.056602        | 0.009215        | -0.015670       | 0.999661        | -0.018645       | 0.325   | 1.697   | 0.964   | display | -1       | 37     | 0.994381|

### Write inference results into ATEK-format csv files


In [ ]:
from atek.evaluation.static_object_detection.obb3_csv_io import AtekObb3CsvWriter
data_dir = "/content/data"
os.makedirs(data_dir, exist_ok = True)

gt_writer = AtekObb3CsvWriter(output_filename = os.path.join(data_dir, "gt_obbs.csv"))
prediction_writer = AtekObb3CsvWriter(output_filename = os.path.join(data_dir, "prediction_obbs.csv"))

for input_data_as_list, output_data_as_list in input_output_data_pairs:
    for single_cubercnn_input, single_cubercnn_output in zip(input_data_as_list, output_data_as_list):
        timestamp_ns = single_cubercnn_input["timestamp_ns"]
        single_cubercnn_output["T_world_camera"] = single_cubercnn_input["T_world_camera"]

        gt_writer.write_from_cubercnn_dict(cubercnn_dict = single_cubercnn_input, timestamp_ns = timestamp_ns)
        prediction_writer.write_from_cubercnn_dict(cubercnn_dict = single_cubercnn_output, timestamp_ns = timestamp_ns)
logger.info("Finished writing obb csv files")

### Call ATEK's benchmarking script to evaluate the results

In [ ]:
benchmarking_command = [
    "python3", f"{atek_src_path}/tools/benchmarking_static_object_detection.py",
    "--pred-csv", f"{data_dir}/prediction_obbs.csv",
    "--gt-csv", f"{data_dir}/gt_obbs.csv",
    "--output-file", f"{data_dir}/atek_metrics.json"
]
return_code = run_command_and_display_output(benchmarking_command)